# UTS PSD

Install Library

In [ ]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import scipy
import zipfile


In [ ]:
!pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d ejlok1/toronto-emotional-speech-set-tess


mkdir: cannot create directory ‘/root/.kaggle’: File exists
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset_path = "/content/drive/MyDrive/PSD/Ektraksi_ciri_audio/AudioTest/TESS Toronto emotional speech set data"

#Subfolder yang digunakan
subfolders = ['OAF_angry',"OAF_disgust",'OAF_Fear','OAF_happy','OAF_neutral','OAF_Pleasant_surprise','OAF_Sad','YAF_angry','YAF_disgust','YAF_fear','YAF_happy','YAF_neutral','YAF_pleasant_surprised','YAF_sad']

## Inisialisasi Ekstraksi Fitur

  - mean_list:
  Menyimpan nilai rata-rata dari suatu data.

  - std_list:
  Menyimpan nilai deviasi standar dari suatu data.

  - maxv_list:
  Menyimpan nilai maksimum dari suatu data.

  - minv_list:
  Menyimpan nilai minimum dari suatu data.

  - median_list:
  Menyimpan nilai median dari suatu data.

  - skew_list:
  Menyimpan nilai skewness dari suatu data.

  - kurt_list:
  Menyimpan nilai kurtosis dari suatu data.

  - modus_list:
  Menyimpan nilai modus dari suatu data (nilai yang paling sering muncul).

  - q1_list:
  Menyimpan nilai kuartil pertama dari suatu data.

  - q3_list:
  Menyimpan nilai kuartil ketiga dari suatu data.

  - iqr_list:
  Menyimpan nilai rentang interkuartil dari suatu data.

  - zcr_mean_list, zcr_median_list, zcr_std_list, zcr_kurt_list, zcr_skew_list:
  Menyimpan statistik yang terkait dengan Zero Crossing Rate (ZCR) dari suatu data audio. ZCR mengukur seberapa sering sinyal audio melintasi nilai nol.

  - rmse_mean_list, rmse_median_list, rmse_std_list, rmse_kurt_list, rmse_skew_list:
  Menyimpan statistik yang terkait dengan Root Mean Square Error (RMSE) dari suatu data audio. RMSE mengukur seberapa besar perbedaan antara sinyal asli dan sinyal yang diprediksi.

In [ ]:
mean_list = []
std_list = []
maxv_list = []
minv_list = []
median_list = []
skew_list = []
kurt_list = []
modus_list = []
q1_list = []
q3_list = []
iqr_list = []
zcr_mean_list = []
zcr_median_list = []
zcr_std_list = []
zcr_kurt_list = []
zcr_skew_list = []
rmse_mean_list = []
rmse_median_list = []
rmse_std_list = []
rmse_kurt_list = []
rmse_skew_list = []

In [ ]:
# data_audio = []
labels = []

for subfolder in subfolders:
    subfolder_path = os.path.join(dataset_path, subfolder)

    # List semua file audio .wav dalam subfolder
    audio_files = [f for f in os.listdir(subfolder_path) if f.endswith('.wav')]

    # Loop melalui setiap file audio .wav
    for audio_file in audio_files:
        audio_file_path = os.path.join(subfolder_path, audio_file)
        x, sr = librosa.load(audio_file_path, sr=None)
        freqs = np.fft.fftfreq(x.size)

        mean_list.append(np.mean(freqs))
        std_list.append(np.std(freqs))
        maxv_list.append(np.amax(freqs))
        minv_list.append(np.amin(freqs))
        median_list.append(np.median(freqs))
        skew_list.append(scipy.stats.skew(freqs))
        kurt_list.append(scipy.stats.kurtosis(freqs))
        modus_list.append(scipy.stats.mode(freqs)[0])
        q1_list.append(np.quantile(freqs, 0.25))
        q3_list.append(np.quantile(freqs, 0.75))
        iqr_list.append(scipy.stats.iqr(freqs))

        zcr = librosa.feature.zero_crossing_rate(x)
        zcr_mean_list.append(np.mean(zcr))
        zcr_median_list.append(np.median(zcr))
        zcr_std_list.append(np.std(zcr))
        zcr_kurt_list.append(scipy.stats.kurtosis(zcr, axis=1)[0])
        zcr_skew_list.append(scipy.stats.skew(zcr, axis=1)[0])

        rmse = librosa.feature.rms(y=x)
        rmse_mean_list.append(np.mean(rmse))
        rmse_median_list.append(np.median(rmse))
        rmse_std_list.append(np.std(rmse))
        rmse_kurt_list.append(scipy.stats.kurtosis(rmse, axis=1)[0])
        rmse_skew_list.append(scipy.stats.skew(rmse, axis=1)[0])
        # data_audio.append(x)
        if subfolder_path.split('_')[-1].lower() == 'surprised':
          labels.append('surprise')
        else:
          labels.append(subfolder_path.split('_')[-1].lower())


## Inisialisasi Dataframe

In [ ]:
df = pd.DataFrame({
    'mean' : mean_list,
    'std' : std_list,
    'max' : maxv_list,
    'min' : minv_list,
    'median' : median_list,
    'modus' : modus_list,
    'skew' : skew_list,
    'kurt' : kurt_list,
    'q1' : q1_list,
    'q3' : q3_list,
    'iqr' : iqr_list,
    'zcr_mean' : zcr_mean_list,
    'zcr_median' : zcr_median_list,
    'zcr_std' : zcr_std_list,
    'zcr_kurt' : zcr_kurt_list,
    'zcr_skew' : zcr_skew_list,
    'rmse_mean' : rmse_mean_list,
    'rmse_median' : rmse_median_list,
    'rmse_std' : rmse_std_list,
    'rmse_kurt' : rmse_kurt_list,
    'rmse_skew' : rmse_skew_list,
    'label' : labels
})
df

,mean,std,max,min,median,modus,skew,kurt,q1,q3,...,zcr_median,zcr_std,zcr_kurt,zcr_skew,rmse_mean,rmse_median,rmse_std,rmse_kurt,rmse_skew,label
0,-1.835027e-18,0.288675,0.499986,-0.499986,0.000000,-0.499986,5.525710e-17,-1.2,-0.249993,0.249993,...,0.065918,0.126751,0.205293,1.319255,0.035394,0.032280,0.026063,-0.812931,0.462762,angry
1,-1.274535e-05,0.288675,0.499975,-0.500000,-0.000013,-0.500000,-7.340884e-17,-1.2,-0.250006,0.249981,...,0.050781,0.123519,2.449860,1.920870,0.043054,0.035144,0.038734,-0.163850,0.883335,angry
2,-1.311510e-05,0.288675,0.499974,-0.500000,-0.000013,-0.500000,-6.391717e-17,-1.2,-0.250007,0.249980,...,0.053711,0.074469,2.841894,1.875801,0.023973,0.018603,0.020482,0.526201,1.040674,angry
3,-1.263456e-05,0.288675,0.499975,-0.500000,-0.000013,-0.500000,1.735302e-16,-1.2,-0.250006,0.249981,...,0.046875,0.080692,6.144893,2.627124,0.034876,0.032349,0.023262,-1.349631,0.232527,angry
4,-1.286339e-05,0.288675,0.499974,-0.500000,-0.000013,-0.500000,5.699133e-18,-1.2,-0.250006,0.249981,...,0.061035,0.078923,5.513442,2.336529,0.031478,0.032358,0.025110,-0.228229,0.572779,angry
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2795,-4.262803e-18,0.288675,0.499991,-0.499991,0.000000,-0.499991,6.662358e-17,-1.2,-0.249995,0.249995,...,0.046875,0.203288,2.999281,2.067435,0.042011,0.038283,0.032375,-1.272148,0.283001,sad
2796,-6.059849e-18,0.288675,0.499991,-0.499991,0.000000,-0.499991,1.737157e-17,-1.2,-0.249996,0.249996,...,0.024414,0.170376,6.357231,2.783468,0.030325,0.031535,0.018902,-0.641601,0.309522,sad
2797,2.651802e-17,0.288675,0.499991,-0.499991,0.000000,-0.499991,-1.888011e-17,-1.2,-0.249996,0.249996,...,0.021973,0.185620,5.642601,2.671948,0.033338,0.034356,0.020127,-1.166536,0.071010,sad
2798,-1.012800e-18,0.288675,0.499991,-0.499991,0.000000,-0.499991,-7.236164e-17,-1.2,-0.249996,0.249996,...,0.032959,0.197696,4.814393,2.496656,0.038239,0.038466,0.022364,-1.025513,-0.012081,sad


## Cek 21 fitur

In [ ]:
df_feature = df[df.columns[0:-1]]
df_feature

,mean,std,max,min,median,modus,skew,kurt,q1,q3,...,zcr_mean,zcr_median,zcr_std,zcr_kurt,zcr_skew,rmse_mean,rmse_median,rmse_std,rmse_kurt,rmse_skew
0,-1.835027e-18,0.288675,0.499986,-0.499986,0.000000,-0.499986,5.525710e-17,-1.2,-0.249993,0.249993,...,0.131496,0.065918,0.126751,0.205293,1.319255,0.035394,0.032280,0.026063,-0.812931,0.462762
1,-1.274535e-05,0.288675,0.499975,-0.500000,-0.000013,-0.500000,-7.340884e-17,-1.2,-0.250006,0.249981,...,0.105919,0.050781,0.123519,2.449860,1.920870,0.043054,0.035144,0.038734,-0.163850,0.883335
2,-1.311510e-05,0.288675,0.499974,-0.500000,-0.000013,-0.500000,-6.391717e-17,-1.2,-0.250007,0.249980,...,0.082624,0.053711,0.074469,2.841894,1.875801,0.023973,0.018603,0.020482,0.526201,1.040674
3,-1.263456e-05,0.288675,0.499975,-0.500000,-0.000013,-0.500000,1.735302e-16,-1.2,-0.250006,0.249981,...,0.070600,0.046875,0.080692,6.144893,2.627124,0.034876,0.032349,0.023262,-1.349631,0.232527
4,-1.286339e-05,0.288675,0.499974,-0.500000,-0.000013,-0.500000,5.699133e-18,-1.2,-0.250006,0.249981,...,0.079410,0.061035,0.078923,5.513442,2.336529,0.031478,0.032358,0.025110,-0.228229,0.572779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2795,-4.262803e-18,0.288675,0.499991,-0.499991,0.000000,-0.499991,6.662358e-17,-1.2,-0.249995,0.249995,...,0.140202,0.046875,0.203288,2.999281,2.067435,0.042011,0.038283,0.032375,-1.272148,0.283001
2796,-6.059849e-18,0.288675,0.499991,-0.499991,0.000000,-0.499991,1.737157e-17,-1.2,-0.249996,0.249996,...,0.086689,0.024414,0.170376,6.357231,2.783468,0.030325,0.031535,0.018902,-0.641601,0.309522
2797,2.651802e-17,0.288675,0.499991,-0.499991,0.000000,-0.499991,-1.888011e-17,-1.2,-0.249996,0.249996,...,0.093970,0.021973,0.185620,5.642601,2.671948,0.033338,0.034356,0.020127,-1.166536,0.071010
2798,-1.012800e-18,0.288675,0.499991,-0.499991,0.000000,-0.499991,-7.236164e-17,-1.2,-0.249996,0.249996,...,0.110387,0.032959,0.197696,4.814393,2.496656,0.038239,0.038466,0.022364,-1.025513,-0.012081


In [ ]:
df_label = df[df.columns[-1]]
df_label

0       angry
1       angry
2       angry
3       angry
4       angry
        ...  
2795      sad
2796      sad
2797      sad
2798      sad
2799      sad
Name: label, Length: 2800, dtype: object

## Split Data

In [ ]:
# import pickle
import joblib

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df_feature, df_label, test_size=0.2, random_state=42)

## Normalisasi standar scaler

In [ ]:
from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()
scaler_z = scalar.fit(x_train)

joblib.dump(scaler_z, 'z-score_scaler.pkl')

['z-score_scaler.pkl']

In [ ]:
x_train_normalized_zero = pd.DataFrame(scaler_z.transform(x_train), columns=x_train.columns)
x_test_normalized_zero = pd.DataFrame(scaler_z.transform(x_test), columns=x_test.columns)

## Z-Score KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=3)
knn_zero = knn.fit(x_train_normalized_zero, y_train)
joblib.dump(knn_zero, "knn_z-score.pkl")

['knn_z-score.pkl']

In [ ]:
from sklearn.metrics import accuracy_score

y_pred_zero_knn = knn_zero.predict(x_test_normalized_zero)

# Menghitung akurasi
accuracy = accuracy_score(y_test, y_pred_zero_knn)
accuracy

0.8267857142857142

## Min Max KNN

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scalar_ = MinMaxScaler()
scaler_mmax = scalar.fit(x_train)
joblib.dump(scaler_mmax, 'mmax_scaler.pkl')

x_train_normalized_mmax = pd.DataFrame(scaler_mmax.transform(x_train), columns=x_train.columns)
x_test_normalized_mmax = pd.DataFrame(scaler_mmax.transform(x_test), columns=x_test.columns)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)
knn_mmax = knn.fit(x_train_normalized_mmax, y_train)

In [ ]:
y_pred_mmax_knn = knn_mmax.predict(x_test_normalized_mmax)

# Menghitung akurasi
accuracy = accuracy_score(y_test, y_pred_mmax_knn)
accuracy

0.8267857142857142

## Grid Mencari K

### Z-Score

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('knn', KNeighborsClassifier())
])

param_grid = {
    'knn__n_neighbors': [i for i in range(1,21)],  # Jumlah tetangga terdekat (K)
}

grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5)

# Jalankan grid search pada data pelatihan
grid_search.fit(x_train_normalized_zero, y_train)

# Cetak parameter terbaik yang ditemukan
print("Z-Score")
print("Parameter terbaik:", grid_search.best_params_)

# Evaluasi model terbaik pada data pengujian
best_model = grid_search.best_estimator_
y_pred = best_model.predict(x_test_normalized_zero)
joblib.dump(knn_zero, "knn_z-score_grid.pkl")

accuracy = accuracy_score(y_test, y_pred)
print("Akurasi pada data pengujian:", accuracy)

Z-Score
Parameter terbaik: {'knn__n_neighbors': 7}
Akurasi pada data pengujian: 0.825


### MinMax

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('knn', KNeighborsClassifier())
])

param_grid = {
    'knn__n_neighbors': [i for i in range(1,21)],  # Jumlah tetangga terdekat (K)
}

grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5)

# Jalankan grid search pada data pelatihan
grid_search.fit(x_train_normalized_mmax, y_train)

# Cetak parameter terbaik yang ditemukan
print("MinMax")
print("Parameter terbaik:", grid_search.best_params_)

# Evaluasi model terbaik pada data pengujian
best_model = grid_search.best_estimator_
y_pred = best_model.predict(x_test_normalized_mmax)
joblib.dump(knn_zero, "knn_mmax_grid.pkl")

accuracy = accuracy_score(y_test, y_pred)
print("Akurasi pada data pengujian:", accuracy)

MinMax
Parameter terbaik: {'knn__n_neighbors': 7}
Akurasi pada data pengujian: 0.825


## PCA

### Z-Score

In [ ]:
from sklearn.decomposition import PCA

pipeline = Pipeline([
    ('pca', PCA()),  # Anda dapat mengatur jumlah komponen PCA di sini
    ('knn', KNeighborsClassifier())
])

param_grid = {
    'pca__n_components': [i for i in range(20, 0, -1)],  # Jumlah komponen PCA
    'knn__n_neighbors': [i for i in range(1, 21)],  # Jumlah tetangga terdekat (K)
}

grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5)

# Jalankan grid search pada data pelatihan
grid_search.fit(x_train_normalized_zero, y_train)

# Cetak parameter terbaik yang ditemukan
print("Zero")
print("Parameter terbaik:", grid_search.best_params_)

# Evaluasi model terbaik pada data pengujian
best_model = grid_search.best_estimator_
y_pred = best_model.predict(x_test_normalized_zero)
joblib.dump(best_model, "knn_zero_grid_PCA.pkl")

accuracy = accuracy_score(y_test, y_pred)
print("Akurasi pada data pengujian:", accuracy)

Zero
Parameter terbaik: {'knn__n_neighbors': 7, 'pca__n_components': 12}
Akurasi pada data pengujian: 0.8267857142857142


In [ ]:
pca = PCA(n_components=20)
zero_pca = pca.fit_transform(x_train_normalized_zero)
joblib.dump(pca, "PCA_zero.pkl")

['PCA_zero.pkl']

### MinMax

In [ ]:
from sklearn.decomposition import PCA

pipeline = Pipeline([
    ('pca', PCA()),  # Anda dapat mengatur jumlah komponen PCA di sini
    ('knn', KNeighborsClassifier())
])

param_grid = {
    'pca__n_components': [i for i in range(20, 0, -1)],  # Jumlah komponen PCA
    'knn__n_neighbors': [i for i in range(1, 21)],  # Jumlah tetangga terdekat (K)
}

grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5)

# Jalankan grid search pada data pelatihan
grid_search.fit(x_train_normalized_mmax, y_train)

# Cetak parameter terbaik yang ditemukan
print("MinMax")
print("Parameter terbaik:", grid_search.best_params_)

# Evaluasi model terbaik pada data pengujian
best_model = grid_search.best_estimator_
y_pred = best_model.predict(x_test_normalized_mmax)
joblib.dump(best_model, "knn_mmax_grid_PCA.pkl")

accuracy = accuracy_score(y_test, y_pred)
print("Akurasi pada data pengujian:", accuracy)

MinMax
Parameter terbaik: {'knn__n_neighbors': 7, 'pca__n_components': 12}
Akurasi pada data pengujian: 0.8267857142857142


In [ ]:
pca = PCA(n_components=20)
zero_pca = pca.fit_transform(x_train_normalized_mmax)
joblib.dump(pca, "PCA_mmax.pkl")

['PCA_mmax.pkl']

In [ ]:
# from sklearn.decomposition import PCA
# from sklearn.metrics import accuracy_score

for i in range(20, 16, -1):
  pca = PCA(n_components=i)
  pca.fit(x_train_normalized_zero)
  pca_train = pca.transform(x_train_normalized_zero)
  pca_test = pca.transform(x_test_normalized_zero)
  print(f'dimensi = {i}')
  for x in range(1,7+1, 2):
    Knn = KNeighborsClassifier(n_neighbors=x)
    Knn.fit(pca_train, y_train)
    y_pred = Knn.predict(pca_test)
    print(f'k = {x} = {accuracy_score(y_test, y_pred)}')



dimensi = 20
k = 1 = 0.8125
k = 3 = 0.8267857142857142
k = 5 = 0.8214285714285714
k = 7 = 0.825
dimensi = 19
k = 1 = 0.8125
k = 3 = 0.8267857142857142
k = 5 = 0.8214285714285714
k = 7 = 0.825
dimensi = 18
k = 1 = 0.8125
k = 3 = 0.8267857142857142
k = 5 = 0.8214285714285714
k = 7 = 0.825
dimensi = 17
k = 1 = 0.8125
k = 3 = 0.8267857142857142
k = 5 = 0.8214285714285714
k = 7 = 0.825
